**Определение стоимости автомобилей**

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. Имеются исторические данные: технические характеристики, комплектации и цены автомобилей. 

Нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Содержание**

1. Подготовка данных
- Загрузка необходимых библиотек и исходных данных
- Предварительное исследование исходных данных
- Исследование и заполнение пропущенных значений
- Удаление дубликатов
- Удаление неинформативных признаков
- Отделение целевого признака
- Применение прямого кодирования для категориальных признаков
- Разбиение датасета на выборки
- Вывод по разделу
     
2. Обучение моделей
- Решающее дерево
- Случайный лес
- Линейная регрессия
- LightGBM
      
3. Анализ моделей
- Решающее дерева
- Случайный лес
- Линейная регрессия
- Градиентный бустинг
- Итоговая таблица
        
4. Вывод по проекту


# Подготовка данных

## Загрузка необходимых библитек и исходных данных

In [1]:
# Для начала, естественно, загружаем необходимые для дальнейшей работы библиотеки и инструменты
# Ради интереса поставим таймер на выполнение всей тетрадки

import pandas as pd
import numpy as np
import sklearn

from datetime import datetime, time

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb

import timeit

from sklearn.model_selection import GridSearchCV

start_time_full = datetime.now()

In [2]:
# Далее загружаем данные и выводим их на экран для визуального ознакомления

df = pd.read_csv('autos.csv')
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


## Предварительное исследование исходных данных

In [3]:
# Для более подробного знакомства с датасетом применим метод info

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
# Не лишним будет воспользоваться и методом describe()

df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [5]:
# Далее считаем количество пропусков в столбцах

df.isnull().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

In [6]:
# Проверим наличие дубликатов

df.duplicated().sum()

4

**Мысли по предварительному изучению данных**

Исходный набор данных содержит 354369 строк (объектов) и 16 столбцов (признкаков). Имеется значительное количество пропущенных данных, несколько дубликатов. 

Более подробно о столбцах можно сказать следующее:

1. DateCrawled — дата скачивания анкеты из базы:
- формат данных Object (в данной задаче можно не переводить в DateTime);
- пропусков нет
- данный столбец, по моему мнению, не несет значимого влияния на стоимость автомобиля. Можно избавиться от него;
2. VehicleType — тип автомобильного кузова:
- формат данных Object;
- имеются пропуски для заполнения (для заполнения используем моду по столбцу);
3. RegistrationYear — год регистрации автомобиля:
- формат данных int (менять не требуется);
- пропусков нет;
4. Gearbox — тип коробки передач:
- формат данных Object (коррекция не требуется);
- пропусков нет;
5. Power — мощность (л.с.):
- формат данных Int (коррекции не требуется);
- пропусков нет;
6. Model — модель автомобиля:
- формат данных Object (коррекция не требуется);
- 19705 пропусков (строки с пропусками в этом столбце можно удалить, так как восстановить значения не представляется возможным, а их количество не является критичным для имеющегося набора данных);
7. Kilometer — пробег (км):
- формат данных int (коррекции не требует);
- пропусков нет;
8. RegistrationMonth — месяц регистрации автомобиля:
- формат данных Int (коррекции не требует);
- пропусков нет;
9. FuelType — тип топлива:
- формат данных Object;
- 32895 пропусков (можно заполнить модой по столбцу);
10. Brand — марка автомобиля:
- формат данных Object (коррекция не требуется);
- пропусков нет;
11. NotRepaired — была машина в ремонте или нет: 
- формат данных Object (коррекция не требуется);
- 71154 пропусков (восстановить значения даже приблизительно не представляется возможным, а параметр достаточно важный для определения цены. Можно заполнить новым классом, который будет обозначать отсутствие данных);
12. DateCreated — дата создания анкеты: 
- формат данных Object (перевести в DateTime можно, но в данной задаче не поможет);
- пропусков нет (Данный столбец, как и самый первый, не несет влияния на стоимость автомобиля. Можно его убить для облегчения датасета);
13. NumberOfPictures — количество фотографий автомобиля:
- формат данных int (коррекция не требуется);
- пропусков нет. Все значения в данном столбце заполнены нулями - признак совершенно не несет никакой смысловой нагрузки. От него нужно избавиться, чтобы не перегружать модель лишними вычислениями;
14. PostalCode — почтовый индекс владельца анкеты (пользователя):
- формат данных int (коррекция не требуется);
- пропусков нет. Представляется, что данный признак не явялется ценообразующим. Можно его зачистить;
15. LastSeen — дата последней активности пользователя:
- формат данных Object (перевести в DateTime, как и ранее, не требуется);
- пропусков нет. Данный столбец, по моему мнению, не несет значимого влияния на стоимость автомобиля. Избавимся от него;
16. Price — цена (евро) - целевой признак:
- формат данных int (корректировки не требует);
- пропусков нет.

## Исследование и заполнение пропущенных значений

In [7]:
# Удалим пропуски в объектах, которые содержат пропуски сразу во всех признаках со значениями NaN

df.drop(df.loc[(df.Gearbox.isna()) & (df.Model.isna()) & (df.VehicleType.isna()) & (df.FuelType.isna()) & (df.NotRepaired.isna())].index, inplace = True)

In [8]:
# Мы не можем узнать из имеющихся данных, была ли машина в ремонте или нет. Поэтому создадим для столбца
# NotRepaired помимо "yes" и "no"  новый класс "unknown", которым и заменим пропуски

df.NotRepaired.fillna('unknown', inplace=True)

In [9]:
# Пропуски в столбце с типами топлива заменим модой по данному столбцу

fuel_mode = df.FuelType.mode()

df.FuelType.fillna(fuel_mode[0], inplace=True)

In [10]:
# То же самое провернем для столбца Gearbox

gear_mode = df.Gearbox.mode()

df.Gearbox.fillna(gear_mode[0], inplace=True)

In [11]:
# Тип машины тяжело определить снаскока. К тому же их очень много, что не дает права заменять значением моды по столбцу.
# В данном случае пропуски заменим новым классом "undefined" 

df.VehicleType.fillna('undefined', inplace = True)

In [12]:
# Определим процентный состав пропусков в столбце Model от общего количества объектов

df.Model.isna().sum()/df.shape[0]*100

4.960639760089966

Пропусков в стобце Model менее 6 %. Вдобавок, если не указана модель автомобиля, то какой смысл вообще выкладывать объявление? Скорее всего, при его создании происходили ошибки, от чего и родились пропуски. Учитывая все это, объекты (строки) с пропусками в этом признаке можно удалить полностью. 

In [13]:
# Удаляем строки с пропусками в признаке Model

df.drop(df[df.Model.isna()].index, axis = 0, inplace=True)

In [14]:
# Снова рассчитываем количество пропусков по столбцам для проверки результатов работы.

df.isnull().sum()

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64

## Удаление дубликатов в данных

In [15]:
# Считаем количество повторов в датасете

df.duplicated().sum()

4

In [16]:
# Удаляем повторяющиеся объекты

df.drop(df[df.duplicated()].index,axis=0,inplace=True)

In [17]:
# Проверяем

df.duplicated().sum()

0

## Удаление неинформативных признаков

In [18]:
# Сразу отбросим столбцы 'NumberOfPictures', 'PostalCode', 'DateCrawled', 'DateCreated', 'LastSeen', которые 
#  являются неинформативными в данной ситуации

df.drop(['NumberOfPictures', 'PostalCode', 'DateCrawled', 'DateCreated', 'LastSeen'], axis = 1, inplace = True)

In [19]:
# Проверяем результат

df.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired
0,480,undefined,1993,manual,0,golf,150000,0,petrol,volkswagen,unknown
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes


## Отделение целевого признака

In [20]:
# В отдельную переменную записываем целевой признак Price

target = df['Price'].reset_index(drop=True)

In [21]:
# Оставшиеся признаки тоже пихаем в отдельный массив

features = df.drop('Price', axis = 1).reset_index(drop=True)

## Применение прямого кодирования для категориальных признаков 

In [22]:
# Для обучения моделей применим прямое кодирование к категориальным признакам. Результат сохраним в отдельный датасет

features_dummies = pd.get_dummies(features, drop_first=True)

In [23]:
# Посмотрим на получившееся безобразие

features_dummies.head()

,RegistrationYear,Power,Kilometer,RegistrationMonth,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,...,Brand_skoda,Brand_smart,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_unknown,NotRepaired_yes
0,1993,0,150000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,2004,163,125000,8,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,2001,75,150000,6,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,2008,69,90000,7,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,1995,102,150000,10,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


## Разбиение датасета на выборки

In [24]:
# Для обучения простых ML моделей разобьем датасет с дамми-признаками
# на тренировочную и тестовую выборки

(features_dummies_train, features_dummies_test, 
 target_train, target_test) = train_test_split(features_dummies, 
                                               target, test_size = 0.2, 
                                               random_state = 12345)

In [25]:
# Снова разобьем данные и создадим отдельные переменные для признаков, где столбцы с данными формата object 
# будут переведены в тип cateegory для работы с ними модели LightGBM

features_train_cat, features_test_cat, target_train_cat, target_test_cat = train_test_split(features, target, test_size = 0.2, random_state = 12345)

In [26]:
# Сохраним в отдельный массив список столбцов, которые нужно перевести в тип категориальных переменных

feats_cat = features_train_cat.dtypes[features_train_cat.dtypes=='object'].index
feats_cat

Index(['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired'], dtype='object')

In [27]:
# Переводим полученные признаки в формат category 

features_train_cat[feats_cat] = (features_train_cat[feats_cat].astype('category'))

features_test_cat[feats_cat] = (features_test_cat[feats_cat].astype('category'))

c:\users\konst\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [28]:
# Проверяем результат 

print(features_train_cat.dtypes, end='\n\n')
print(features_test_cat.dtypes)

VehicleType          category
RegistrationYear        int64
Gearbox              category
Power                   int64
Model                category
Kilometer               int64
RegistrationMonth       int64
FuelType             category
Brand                category
NotRepaired          category
dtype: object

VehicleType          category
RegistrationYear        int64
Gearbox              category
Power                   int64
Model                category
Kilometer               int64
RegistrationMonth       int64
FuelType             category
Brand                category
NotRepaired          category
dtype: object


## Вывод по разделу

На данном этапе работы были осуществлены действия по подготовке данных к дальнейшему анализу. А точнее, совершено следующее:
    
- удалось загрузить необходимые инструменты и библиотеки, "залить" в тетрадку исходный датасет;
- был выполнен поиск и заполнение пропусков в признаках (некоторые пришлось и удалить), аннигиляция дубликатов среди объектов;
- датасет был самым жестоким образом уменьшен в размере путем "убивания" неинформативных признаков (даты заведения страницы, последнего посещения, загрузки объявления), что позволит значительно облегчить и ускорить работу моделей машинного обучения и уменьшить объемы используемой памяти
- от основного массива данных отделен целевой признак;
- применено прямое кодирование категориальных признаков. 
- созданы отдельные переменные для признаков, где столбцы с данными формата object переведены в тип cateegory для работы с ними модели LightGBM

# Обучение моделей

## Решающее дерево

In [29]:
%%time

# Начнем с самого простого - обучающего дерева. Для поиска оптимальных параметров начнем осваивать GridSearch

model_tree = DecisionTreeRegressor()

param_grid = {
    'max_depth': [10,15],
    'min_samples_leaf': [15, 20],
}

gs = GridSearchCV(estimator=model_tree, param_grid=param_grid,  cv=3, n_jobs=-1, 
                  scoring='neg_mean_squared_error', verbose=2)

model_tree_fitted = gs.fit(features_dummies_train, target_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Wall time: 18.4 s


In [30]:
# Выведем луйший результат метрики и подобранные гиперпараметры модели

RMSE_tree_train = (model_tree_fitted.best_score_*(-1))**0.5 
print(RMSE_tree_train)
print(model_tree_fitted.best_params_)

1969.2022257992053
{'max_depth': 15, 'min_samples_leaf': 15}


## Случайный лес

In [31]:
%%time
start_time = datetime.now() 

# Подберем оптимальные параметры для обучения случайного леса

model_forest = RandomForestRegressor()

param_grid = {
    'max_depth': [10,15],
    'n_estimators': [15, 20],
}

gs = GridSearchCV(estimator=model_forest, param_grid=param_grid,  cv=3, n_jobs=-1, 
                  scoring='neg_mean_squared_error', verbose=10)

model_forest_fitted = gs.fit(features_dummies_train, target_train)

time_forest_GS = datetime.now() - start_time
print(str(time_forest_GS))

Fitting 3 folds for each of 4 candidates, totalling 12 fits
0:03:16.736118
Wall time: 3min 16s


In [32]:
# Выведем луйший результат метрики и подобранные гиперпараметры модели

RMSE_forest_train = (model_forest_fitted.best_score_*(-1))**0.5 
print(RMSE_forest_train)
print(model_forest_fitted .best_params_)

1836.5091998676603
{'max_depth': 15, 'n_estimators': 20}


## Линейная регрессия

In [33]:
# Обучаем модель линейной регрессии 

start_time = datetime.now()

model_lin_regr = LinearRegression()
model_lin_regr.fit(features_dummies_train, target_train)

time_LR_fitting = datetime.now() - start_time
print(str(time_LR_fitting))

0:00:03.899037


## LightGBM

In [34]:
%%time

# Теперь подключим к работе модель градиентного бустинга

train_dataset = lgb.Dataset(features_train_cat, label=target_train_cat)
test_dataset = lgb.Dataset(features_test_cat, label=target_test_cat)

parameters = {'max_depth': np.arange(10, 15),
              'n_estimators': np.arange(15, 20)
             }

gs = GridSearchCV(lgb.LGBMRegressor(random_state=12345), 
                  parameters,
                  cv=3,
                  scoring='neg_mean_squared_error')

# обучение
gs.fit(features_train_cat, target_train_cat)
# просмотр лучших параметров
gs.best_params_

Wall time: 15.8 s


{'max_depth': 10, 'n_estimators': 19}

In [35]:
# просмотр лучших параметров
gs.best_params_

{'max_depth': 10, 'n_estimators': 19}

In [36]:
# Посмотрим на метрику RMSE при полученных гиперпараметрах модели

RMSE_LightGbm_train = ((gs.best_score_)*-1)**0.5
print('Метрика RMSE модели LightGBM: {:.2f}'.format(RMSE_LightGbm_train))

Метрика RMSE модели LightGBM: 2089.85


# Анализ моделей

## Решающее дерево

In [37]:
%%time
start_time = datetime.now()

# Дадим решающему дереву с GS-параметрами проявить себя в предасказаниях! Обучаем модель.

tree = DecisionTreeRegressor(max_depth = model_tree_fitted.best_params_['max_depth'],
                             min_samples_leaf=model_tree_fitted.best_params_['min_samples_leaf'], random_state=12345)

tree.fit(features_dummies_train, target_train)


time_tree_fitting = datetime.now() - start_time
print(str(time_tree_fitting))

0:00:05.807089
Wall time: 5.81 s


In [38]:
# Делаем предсказания

start_time = datetime.now()

predicts = tree.predict(features_dummies_test)

time_tree_test = datetime.now() - start_time
print(str(time_tree_test))

0:00:00.070338


In [39]:
# Рассчитываем метрику RMSE

rmse_tree_test = mean_squared_error(target_test, predicts)**0.5

print('RMSE_DecisionTreeRegressor: {:.2f}'.format(rmse_tree_test))

RMSE_DecisionTreeRegressor: 1912.42


## Случайный лес

In [40]:
%%time
start_time = datetime.now()

# На этом этапе обучаем случайны лес с подобранными ранее гиперпараметрами

forest = RandomForestRegressor(random_state = 12345, n_estimators = model_forest_fitted .best_params_['n_estimators'], 
                               max_depth = model_forest_fitted .best_params_['max_depth'])

forest.fit(features_dummies_train, target_train)

time_forest_fitting = datetime.now() - start_time
print(str(time_forest_fitting))

0:01:14.279106
Wall time: 1min 14s


In [41]:
# Запускаем полученную модель на предсказания
start_time = datetime.now()

predicts = forest.predict(features_dummies_test)

time_forest_test = datetime.now() - start_time
print(str(time_forest_test))

0:00:00.231139


In [42]:
# Считаем RMSE для результатов модели

rmse_forest_test = mean_squared_error(target_test, predicts)**0.5

print(rmse_forest_test)

1801.3800238227657


## Линейная регрессия

In [43]:
# Линейная регрессия была обучена ранее (см. п. 2.3). Теперь делаем предсказания.

start_time = datetime.now()

predicts = model_lin_regr.predict(features_dummies_test)

time_LR_test = datetime.now() - start_time
print(str(time_LR_test))

0:00:00.121935


In [44]:
# Считаем RMSE для линейной регрессии

rmse_LR_test = mean_squared_error(target_test, predicts)**0.5
print('RMSE_LinearRegression: {}'.format(rmse_LR_test))

RMSE_LinearRegression: 3138.202357482589


## Градиентный бустинг LightGBM

In [45]:
# Обучаем модель градиентного бустинга

start_time = datetime.now()

model_lgb = (lgb.LGBMRegressor(max_depth=gs.best_params_['max_depth'], 
                           n_estimators=gs.best_params_['n_estimators'],
                           random_state=12345)
             .fit(features_train_cat, target_train_cat)
            )

time_lightGBM_fitting = datetime.now() - start_time
print(str(time_lightGBM_fitting))

0:00:00.184836


In [46]:
# Делаем предсказания

start_time = datetime.now()

predictions_test = model_lgb.predict(features_test_cat)

time_lightGBM_test = datetime.now() - start_time
print(str(time_lightGBM_test))

0:00:00.032029


In [47]:
mse_lightGBM_test = mean_squared_error(target_test_cat, predictions_test)
rmse_lightGBM_test = mse_lightGBM_test ** 0.5
print('RMSE_lightGBM: {}'.format(rmse_lightGBM_test))

RMSE_lightGBM: 2074.6860517778673


## Итоговая таблица

In [48]:
# Сгруппируем результаты в отдельную таблиуцу

table_final = pd.DataFrame({'Модель': ['LinearRegression', 'DecisionTreeRegressor', 'RandomForestRegressor', 'LightGBM'],
                            'Время обучения': [time_LR_fitting, time_tree_fitting, time_forest_fitting, time_lightGBM_fitting],                           
                            
                            'Время предсказания': [time_LR_test, time_tree_test, time_forest_test, time_lightGBM_test],
                            
                            'Общее время работы': [str(time_LR_fitting+time_LR_test), 
                                                   str(time_tree_fitting+time_tree_test), 
                                                   str(time_forest_fitting+time_forest_test), 
                                                   str(time_lightGBM_fitting+time_lightGBM_fitting)], 
                           'RMSE_test': [rmse_LR_test, rmse_tree_test, rmse_forest_test, rmse_lightGBM_test]})

In [49]:
# Выведем результаты для ознакомления

table_final

,Модель,Время обучения,Время предсказания,Общее время работы,RMSE_test
0,LinearRegression,0 days 00:00:03.899037,0 days 00:00:00.121935,0:00:04.020972,3138.202357
1,DecisionTreeRegressor,0 days 00:00:05.807089,0 days 00:00:00.070338,0:00:05.877427,1912.420431
2,RandomForestRegressor,0 days 00:01:14.279106,0 days 00:00:00.231139,0:01:14.510245,1801.380024
3,LightGBM,0 days 00:00:00.184836,0 days 00:00:00.032029,0:00:00.369672,2074.686052


In [50]:
# Оценим расчета всего проекта

time_total = datetime.now() - start_time_full
print('Время выполнения всей тетрадки:', str(time_total))

Время выполнения всей тетрадки: 0:05:19.458715


## Вывод по проекту

В ходе данной работы были закреплены навыки подготовки данных к анализу (осмотр датасета, поиски заполнение пропусков, удаление дублей, разделение выборки на тренировочную и тестовую, применение прямого кодирования). 
    
Далее с помощью гридсерча были подобраны оптимальные параметры для моделей решающего дерева, случайного леса, градиентного бустинга LightGBM, а также обучена модель линейной регрессии.
    
На заключительном этапе подобранные гиперпараметры были применены для обучения перечисленных моделей, а затем формирования с помощью них предсказаний целевого признака и расчет метрики RMSE.
    
По результатам расчетов можно сделать следующие заключения (время обучения и предсказания я сложил, чтобы получить единую метрику - общее вреям работы модели):
    
1. линейная регрессия значительно уступает остальным моделям по качеству, а также занимает 3 место из 4 по скорости. Ее даже не рассматриваем для использования в работе;
2. случайный лес рвет всех по качеству, но скорость его обучения непозволительно маленькая. Если нужно только качество, то лучше ее сейчас не найти, но в остальном - минус;
3. градиентный бустинг значительно обходит по скорости модель случайного леса, но отстает от нее по RMSE. Вроде бы неплохо, но...
4. решающее древо показало лучшие результаты по всем фронтам! Скорость лучше всех (8 секунд, соревнуется только с градиентным бустингом), RMSE лучше бустинга, но немного уступает лесу, хотя и не критично (выигрыш по времени перевешивает этот минус). 
    
Таким образом, для использования в работе я бы рекомендовал модель решающего дерева. 